In [ ]:
from sklearn.metrics import accuracy_score, classification_report
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from efficientnet_pytorch import EfficientNet
from PIL import Image
import os
import re
import numpy as np
import random
import pickle

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
input_size = 224
model_path = './models/efficientnet-b3-3000.pth'

In [ ]:
with open('idx2class3000.pkl', 'rb') as f:
    class_names = pickle.load(f)

model_test = EfficientNet.from_pretrained('efficientnet-b3')
num_ftrs = model_test._fc.in_features

model_test._fc  = nn.Linear(num_ftrs, len(class_names))

model_test.to(device)

model_test.load_state_dict(torch.load(model_path))
model_test.eval()

In [ ]:
data_transforms = transforms.Compose([transforms.Resize((input_size, input_size)),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.726, 0.686, 0.695], [0.205, 0.210, 0.183])])

In [ ]:
def inference_model(model, input_image_path):
    was_training = model.training
    model.eval()

    with torch.no_grad():
        with Image.open(input_image_path) as input_image:
            resized_image = data_transforms(input_image).unsqueeze(0).to(device)
            
            output = model(resized_image)
            _, pred = torch.max(output, 1)
            softmax = nn.Softmax(dim=1)
            prob_output = softmax(output)
            prob, pred = torch.max(prob_output, 1)
        model.train(mode=was_training)
    
    return pred.item(), prob.item(), np.array(prob_output.cpu())[0]

In [ ]:
from sklearn.metrics import f1_score
import pandas as pd

test_dir = './model_data/test/'
test_img_paths = []
for root, dirs, files in os.walk(test_dir):
    for name in files:
        label = root.replace(test_dir, '')
        test_img_paths.append((label, os.path.join(root, name)))
random.shuffle(test_img_paths)

thresholds = np.array([0.0, 0.3, 0.5, 0.7, 0.8])
test_preds = [[] for _ in range(len(thresholds))]
test_labels = []

for (label, img_path)  in test_img_paths:
    pred, max_prob, probs = inference_model(model_test, img_path)
    for i in range(len(thresholds)):
        if max_prob < thresholds[i]:
            test_preds[i].append('isnull')
        else:
            test_preds[i].append(class_names[pred])
    test_labels.append(label)

for i in range(len(thresholds)):
    report = classification_report(test_labels, test_preds[i], output_dict=True)
    df = pd.DataFrame(report).transpose()
    df.to_csv('./result/model_3000_threshold_{}.csv'.format(str(thresholds[i]).replace('.', '')))

In [ ]:
test_dir = './esun_data/test/'
test_img_paths = []
for root, dirs, files in os.walk(test_dir):
    for name in files:
        label = root.replace(test_dir, '')
        test_img_paths.append((label, os.path.join(root, name)))
random.shuffle(test_img_paths)

with open('wordset800.txt', 'r') as f:
    wordset = f.read().split('\n')

thresholds = np.array([0.0, 0.3, 0.5, 0.7, 0.8])
test_preds = [[] for _ in range(len(thresholds))]
test_labels = []

for (label, img_path)  in test_img_paths:
    pred, max_prob, probs = inference_model(model_test, img_path)

    for i in range(len(thresholds)):
        if max_prob < thresholds[i]:
            test_preds[i].append('isnull')
        else:
            if class_names[pred] in wordset:
                test_preds[i].append(class_names[pred])
            else:
                test_preds[i].append('isnull')
    test_labels.append(label)

for i in range(len(thresholds)):
    report = classification_report(test_labels, test_preds[i], output_dict=True)
    df = pd.DataFrame(report).transpose()
    df.to_csv('./result/esun_data_model_3000_threshold_{}.csv'.format(str(thresholds[i]).replace('.', '')))